In [ ]:
# Importa bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.integrate import odeint

In [ ]:
def set_axes_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)
    
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([0, z_middle + plot_radius])

In [ ]:
d_ar = 1.2
g = 9.8
r = (0.66/(2*pi))
m = 0.27
cl = 1
A = pi*(r**2)
Cd = 0.077

quadra_x = [0, 18, 18, 0, 0]
quadra_y = [-4.5, -4.5, 4.5, 4.5, -4.5]
quadra_z = [0, 0, 0, 0, 0]

rede_x = [9, 9, 9, 9, 9]
rede_y = [-4.5, -4.5, 4.5, 4.5, -4.5]
rede_z = [1.43, 2.43, 2.43, 1.43, 1.43]

In [ ]:
tetha = radians(90) # angulo vertical
alpha = radians(0) # angulo horizontal

x0 = 0
y0 = 0
z0 = 3

v0 = 126.3/3.6
vx0 = v0*sin(tetha)*cos(alpha)
vy0 = v0*sin(tetha)*sin(alpha)
vz0 = v0*cos(tetha)

w = 2*(2*pi)
wx = -w*cos(tetha)*cos(alpha)
wy = -w*cos(tetha)*sin(alpha)
wz = w*sin(tetha)

w_vetor = np.array([wx, wy, wz])

info_0 = [x0, y0, z0, vx0, vy0, vz0]

In [ ]:
dt = 1e-3
tMax = 1+dt
t_lista = np.arange(0, tMax, dt)

def modelo(info, t_lista):
    
    x = info[0]
    y = info[1]
    z = info[2]
    vx = info[3]
    vy = info[4]
    vz = info[5]
    V_vetor = np.array([vx, vy, vz])
    
    v_xy = sqrt(vx**2 + vy**2)
    v = sqrt(v_xy**2 + vz**2)

    cos_tetha = vz/v
    sen_tetha = v_xy/v
    cos_alpha = vx/v_xy
    sen_alpha = vy/v_xy
    
    Fa = 0.5*d_ar*A*Cd*(v**2)
    Fa_x = -Fa*sen_tetha*cos_alpha
    Fa_y = -Fa*sen_tetha*sen_alpha
    Fa_z = -Fa*cos_tetha
    
    Fmag = cl*pi*d_ar*(r**3)*np.cross(w_vetor, V_vetor)
    Fmag_x = Fmag[0]
    Fmag_y = Fmag[1]
    Fmag_z = Fmag[2]
    
    dVx_dt = (Fa_x + Fmag_x)/m
    dVy_dt = (Fa_y + Fmag_y)/m
    dVz_dt = (-(m*g) + Fa_z + Fmag_z)/m
    
    if z <= 0:
        return [0, 0, 0, 0, 0, 0]
    else:
        return [vx, vy, vz, dVx_dt ,dVy_dt ,dVz_dt]

In [ ]:
infos_Odeint = odeint(modelo, info_0, t_lista)

posX = list(infos_Odeint[:,0])
posY = list(infos_Odeint[:,1])
posZ = list(infos_Odeint[:,2])

get_ipython().magic('matplotlib qt5')
import matplotlib.patches as patches
from matplotlib import animation
%matplotlib qt5

# Defindo o tempo decorrido para a bola bater na rede
# Dado: 0.267 segundos
for i, t in enumerate(t_lista):
    if posX[i] >= 9:
        tempo = t
        break 

ax = plt.axes(projection='3d')
ax.plot3D(rede_x, rede_y, rede_z, color='grey') # rede
ax.plot3D(quadra_x, quadra_y, quadra_z, color='black') # quadra
ax.plot3D(posX[0], posY[0], posZ[0], 'ro') # Bola

ax.plot3D(posX, posY, posZ, color='orange', label=f"{tempo} segundos")
plt.title("Trajetória da Bola de Volei")
plt.grid()
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.legend()
set_axes_equal(ax)
plt.show()

In [ ]:
# lista_v = np.arange(100, 150, 0.5) # velocidades iniciais [km/h]
# lista_ang_h = np.arange(-3, 9, 0.5) # angulos verticais iniciais [°]

# for ang_h in lista_ang_h:
#     for v in lista_v:

#         angulo_horizontal = radians(ang_h)

#         v0 = v/3.6
#         vx0 = v0*cos(angulo_vertical)*cos(angulo_horizontal)
#         vy0 = v0*cos(angulo_vertical)*sin(angulo_horizontal)
#         vz0 = -v0*sin(angulo_vertical)

#         w = 2*(2*pi)
#         wx = w*sin(angulo_vertical)*cos(90-angulo_horizontal)
#         wy = -w*sin(angulo_vertical)*sin(90-angulo_horizontal)
#         wz = w*cos(angulo_vertical)

#         w = np.array([wx, wy, wz])

#         info_0 = [x0, y0, z0, vx0, vy0, vz0]

#         infos_Odeint = odeint(modelo, info_0, t_lista)

#         posX = list(infos_Odeint[:,0])
#         posY = list(infos_Odeint[:,1])
#         posZ = list(infos_Odeint[:,2])

#         rede = False
#         cor = "ro"
#         for i, t in enumerate(t_lista):
#             if not rede and (posX[i] >= 9 and posZ[i] >= 2.43): # passou da rede
#                 cor = "bo"
#                 rede = True
#             if rede and (posX[i] <= 18 and posZ[i] <= 0): # caiu dentro da quadra
#                 cor = 'go'
#                 break

#         plt.plot(ang_v, v, cor)
# plt.title("Angulo Vertical [°] X Velocidade Inicial [km/h]")
# plt.xlabel('Angulo Vertical [°]')
# plt.ylabel('Velocidade Inicial [km/h]')
# plt.grid()
# plt.legend()
# plt.show()
